In [3]:
import requests
import json
import pandas as pd
import os

In [ ]:
# --------------------
# Useful functions
# --------------------
def write_list(l,file_path,header, initial_pos= 0):
    f = open(file_path,"w+")
    
    if len(l) > 0:
        #header
        if header != None:
            f.write(",".join(header)+"\n")

        #content
        for l_index in range(initial_pos,len(l)):
            str_row = []
            for k in header:
                if k in l[l_index]:
                    str_row.append('"'+str(l[l_index][k])+'"')
            f.write(",".join(str_row)+"\n")
    else:
        f.write("")
    
    
def normalize_mag(l,reuslts_format):
    
    def normalize_atts(l):
        MAG_EXTRA_ATTS = ["logprob","prob"]
        for elem in l:                    
            for mag_extra_att in MAG_EXTRA_ATTS:
                if mag_extra_att in elem:
                    del elem[mag_extra_att]
        return l
    
    if len(l) > 0:
        #normalize elem att
        for elem in l:
                for att in reuslts_format:
                    #check if is a nested att
                    if(att[0] in elem):
                        if len(att) > 1:
                            nested_att_list = []
                            for nested_obj in elem[att[0]]:
                                nested_att_list.append(nested_obj[att[1]])

                            elem[att[0]] = ";; ".join(nested_att_list)
                            
                            #assign a new key name (nested)
                            elem[str(att[0])+"."+str(att[1])] = elem[att[0]]
                            del elem[att[0]]
                        
                        elif isinstance(elem[att[0]], list):
                            elem[att[0]] = ";; ".join(elem[att[0]])
                         
                    #set null attributes to ""
                    else:
                        elem[att[0]] = ""
        
    return normalize_atts(l)

In [ ]:
# Read the Crossref DH articles (the min file generated with "acdh_and_cr_dump.ipynb" script)
CR_File = "../data/gen/dh_in_crossref_min.json"
CR_elems = {}
with open(CR_File) as json_file:
    CR_elems = json.load(json_file)

In [ ]:
## MICROSOFT ACADEMIC GRAPH APROACH
#https://www.microsoft.com/en-us/research/project/academic-knowledge/

#AUTH
USERNAME = "<MAG-USERNAME>"
PKEY = "<MAG-PKEY>"
SKEY = "<MAG-SKEY>"

#CALLS
MAG_API_QUERY = "https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate?expr="
HEADERS = {
    "Host": "api.labs.cognitive.microsoft.com",
    "Ocp-Apim-Subscription-Key": PKEY
}

#NOTE Write the &attributes in sorted order
EXPR = {
    "title":"Ti='<VAR>'",
    "doi":"DOI='<VAR>'",
    "author": "Composite(AA.AuN='<VAR>')"
}
RESULTS ={
    "author": "&attributes=AA.AuN,AA.AuId,AA.DAfN",
    "affiliation": "&attributes=AA.DAfN,AW,DOI,F.FN",
    "citation_context": "&attributes=CitCon"
}    

# TESTING MAG: COMMENT
#r = requests.get(MAG_API_QUERY + EXPR["doi"].replace("<VAR>","10.1007/s11192-019-03217-6")+ RESULTS["citation_context"], headers=HEADERS) 
#json.loads(r.text)

In [ ]:
MAG_CSV_OUTPUT_FILE = "../data/gen/dh_in_mag.json"
MAG_RESULTS = []
#MAG_CSV_HEADER = RESULTS["affiliation"].replace("&attributes=","").split(",")
#MAG_CSV_FORMAT = [att.split(".") for att in MAG_CSV_HEADER]

for doi_k in CR_elems:
    r = requests.get( MAG_API_QUERY + EXPR["doi"].replace("<VAR>",doi_k)+ RESULTS["affiliation"], headers=HEADERS) 
    r_json = json.loads(r.text)
    if len(r_json["entities"]) > 0:    
        MAG_RESULTS.append(r_json["entities"][0])

with open(MAG_CSV_OUTPUT_FILE, 'w') as fp:
    json.dump(MAG_RESULTS, fp)